In [1]:
!pip install yfinance ipywidgets --quiet

import numpy as np
from scipy.stats import norm
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D  # needed for 3D plots
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
from IPython.display import display

# Make plots a bit bigger
plt.rcParams["figure.figsize"] = (8, 5)


# ====================================================
# Core Black–Scholes Functions
# ====================================================

def black_scholes_call_put(S, K, r, sigma, T):
    """
    Black–Scholes price for European call & put options.
    S: spot, K: strike, r: risk-free rate, sigma: vol, T: maturity (years)
    """
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return np.nan, np.nan, np.nan, np.nan

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    call = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    put  = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return call, put, d1, d2


def greeks(S, K, r, sigma, T, call=True):
    """
    Return: (Delta, Gamma, Theta, Vega, Rho)
    """
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return (np.nan,)*5

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    pdf_d1 = norm.pdf(d1)

    # Delta
    delta = norm.cdf(d1) if call else norm.cdf(d1) - 1

    # Gamma (same for both)
    gamma = pdf_d1 / (S * sigma * np.sqrt(T))

    # Vega
    vega = S * pdf_d1 * np.sqrt(T)

    # Theta
    if call:
        theta = -(S * pdf_d1 * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    else:
        theta = -(S * pdf_d1 * sigma) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)

    # Rho
    rho = K * T * np.exp(-r * T) * norm.cdf(d2) if call else -K * T * np.exp(-r * T) * norm.cdf(-d2)

    return delta, gamma, theta, vega, rho


# ====================================================
# Interactive Sliders with ipywidgets
#    - Move S, K, σ, T, r and see:
#      * Payoff vs S
#      * Call & Put price vs S
#      * Greeks at chosen point
# ====================================================

def interactive_black_scholes(S=100, K=100, r=0.05, sigma=0.2, T=0.5):
    # Underlying price range around S
    S_min = max(1, S * 0.5)
    S_max = S * 1.5
    S_range = np.linspace(S_min, S_max, 200)

    call_vals = []
    put_vals = []
    for S_grid in S_range:
        c, p, d1, d2 = black_scholes_call_put(S_grid, K, r, sigma, T)
        call_vals.append(c)
        put_vals.append(p)

    call_vals = np.array(call_vals)
    put_vals = np.array(put_vals)

    # Current point (S,K) values
    call_price, put_price, d1_cur, d2_cur = black_scholes_call_put(S, K, r, sigma, T)
    delta_c, gamma_c, theta_c, vega_c, rho_c = greeks(S, K, r, sigma, T, call=True)
    delta_p, gamma_p, theta_p, vega_p, rho_p = greeks(S, K, r, sigma, T, call=False)

    # --- Plot prices vs S ---
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(S_range, call_vals, label="Call price")
    plt.plot(S_range, put_vals, label="Put price")
    plt.axvline(S, linestyle="--", label="Current S")
    plt.axhline(0, color="black", linewidth=0.5)
    plt.title("Call & Put Price vs Underlying Price S")
    plt.xlabel("Underlying Price S")
    plt.ylabel("Option Price")
    plt.legend()
    plt.grid(True)

    # --- Payoff vs S at maturity (for reference) ---
    payoff_call = np.maximum(S_range - K, 0)
    payoff_put = np.maximum(K - S_range, 0)

    plt.subplot(1, 2, 2)
    plt.plot(S_range, payoff_call, label="Call payoff (at expiry)")
    plt.plot(S_range, payoff_put, label="Put payoff (at expiry)")
    plt.axvline(S, linestyle="--", label="Current S")
    plt.axvline(K, linestyle=":", label="Strike K")
    plt.title("Payoff at Expiry vs S (not discounted)")
    plt.xlabel("Underlying Price S")
    plt.ylabel("Payoff")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # --- Print current option metrics ---
    print("=== Current Parameters ===")
    print(f"S = {S:.2f}, K = {K:.2f}, r = {r:.4f}, sigma = {sigma:.4f}, T = {T:.4f} years")
    print("\nPrices at current S:")
    print(f"Call Price (BSM): {call_price:.4f}")
    print(f"Put  Price (BSM): {put_price:.4f}")
    print(f"d1 = {d1_cur:.4f}, d2 = {d2_cur:.4f}")

    print("\nCall Greeks: (Delta, Gamma, Theta, Vega, Rho)")
    print(f"({delta_c:.4f}, {gamma_c:.6f}, {theta_c:.4f}, {vega_c:.4f}, {rho_c:.4f})")

    print("\nPut Greeks: (Delta, Gamma, Theta, Vega, Rho)")
    print(f"({delta_p:.4f}, {gamma_p:.6f}, {theta_p:.4f}, {vega_p:.4f}, {rho_p:.4f})")


print("👇 Use the sliders below to interact with S, K, r, sigma, T")

interact(
    interactive_black_scholes,
    S=FloatSlider(description="Spot S", min=10, max=300, step=5, value=100),
    K=FloatSlider(description="Strike K", min=10, max=300, step=5, value=100),
    r=FloatSlider(description="r", min=-0.01, max=0.2, step=0.005, value=0.05),
    sigma=FloatSlider(description="sigma", min=0.05, max=1.0, step=0.05, value=0.2),
    T=FloatSlider(description="T (years)", min=0.05, max=2.0, step=0.05, value=0.5)
);



# ====================================================
# Market vs Black–Scholes: Error Plots
#    - Pick ticker & expiry
#    - Compare market call prices vs BSM with single sigma
#    - Show error (market - model)
# ====================================================

def market_vs_bsm(ticker="AAPL", r=0.04):
    print(f"\nFetching data for {ticker} ...")
    tk = yf.Ticker(ticker)

    hist = tk.history(period="1d")
    if hist.empty:
        print("No historical data. Check ticker / connection.")
        return

    S0 = float(hist["Close"].iloc[-1])

    if not tk.options:
        print("No options available for this ticker.")
        return

    expiry = tk.options[0]  # first expiry
    chain = tk.option_chain(expiry)
    calls = chain.calls.copy()

    if calls.empty:
        print("No calls for this expiry.")
        return

    # Clean data
    calls = calls.dropna(subset=["lastPrice", "impliedVolatility", "strike"])
    if calls.empty:
        print("No clean call data.")
        return

    # Use a single "constant" volatility = median IV (for all strikes)
    sigma_const = float(calls["impliedVolatility"].median())

    today = dt.datetime.today().date()
    expiry_date = dt.datetime.strptime(expiry, "%Y-%m-%d").date()
    T = (expiry_date - today).days / 365.0
    if T <= 0:
        print("Expiry is in the past or today. Choose different expiry.")
        return

    K_arr = calls["strike"].values
    market_prices = calls["lastPrice"].values

    model_prices = []
    for K in K_arr:
        c, p, d1, d2 = black_scholes_call_put(S0, K, r, sigma_const, T)
        model_prices.append(c)
    model_prices = np.array(model_prices)

    errors = market_prices - model_prices

    # --- Plot comparison ---
    fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

    axes[0].plot(K_arr, market_prices, "o-", label="Market Call Price")
    axes[0].plot(K_arr, model_prices, "s--", label=f"BSM (σ={sigma_const:.2f})")
    axes[0].set_ylabel("Price")
    axes[0].set_title(f"{ticker} Calls – Market vs Black–Scholes\nExpiry: {expiry}, S0={S0:.2f}, T={T:.3f} years")
    axes[0].legend()
    axes[0].grid(True)

    axes[1].axhline(0, color="black", linewidth=0.8)
    axes[1].plot(K_arr, errors, "o-", label="Market - BSM")
    axes[1].set_xlabel("Strike K")
    axes[1].set_ylabel("Error")
    axes[1].set_title("Pricing Error (Market - Model)")
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()

    print(f"Using constant sigma (median IV) = {sigma_const:.4f}")
    print(f"Mean abs error: {np.mean(np.abs(errors)):.4f}")


print("\n👇 Choose a ticker to compare market vs BSM (first expiry)")
interact(
    market_vs_bsm,
    ticker=Dropdown(
        options=["AAPL", "MSFT", "TSLA", "GOOGL", "AMZN"],
        value="AAPL",
        description="Ticker"
    ),
    r=FloatSlider(description="r", min=0.0, max=0.10, step=0.005, value=0.04),
);



# ====================================================
# Monte Carlo Simulation vs BSM
#    - Simulate stock paths under GBM
#    - Price call with discounted payoff
#    - Compare Monte Carlo price vs BSM
# ====================================================

def monte_carlo_call_price(S, K, r, sigma, T, n_paths=10000, seed=0):
    """
    Simple 1-step Monte Carlo for European call.
    """
    if seed is not None:
        np.random.seed(seed)

    # Sample Z ~ N(0,1)
    Z = np.random.randn(n_paths)
    # Terminal prices under GBM
    ST = S * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    # Payoff
    payoff = np.maximum(ST - K, 0.0)
    # Discount
    price_mc = np.exp(-r * T) * np.mean(payoff)
    std_err = np.exp(-r * T) * np.std(payoff) / np.sqrt(n_paths)
    return price_mc, std_err, ST, payoff


def mc_vs_bsm(S=100, K=100, r=0.05, sigma=0.2, T=1.0, n_paths=20000):
    call_bsm, _, _, _ = black_scholes_call_put(S, K, r, sigma, T)

    price_mc, std_err, ST, payoff = monte_carlo_call_price(S, K, r, sigma, T, n_paths=n_paths, seed=42)

    print("=== Monte Carlo vs Black–Scholes (Call) ===")
    print(f"S={S}, K={K}, r={r}, sigma={sigma}, T={T}, paths={n_paths}")
    print(f"BSM Call Price    : {call_bsm:.4f}")
    print(f"Monte Carlo Price : {price_mc:.4f} ± {1.96*std_err:.4f} (95% CI)")
    print(f"Abs Error         : {abs(price_mc - call_bsm):.4f}")

    # Histogram of ST
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.hist(ST, bins=60, alpha=0.7)
    plt.axvline(S, linestyle="--", label="Initial S")
    plt.axvline(K, linestyle=":", label="Strike K")
    plt.title("Distribution of Terminal Price $S_T$")
    plt.xlabel("$S_T$")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)

    # Histogram of discounted payoff
    disc_payoff = np.exp(-r * T) * payoff
    plt.subplot(1, 2, 2)
    plt.hist(disc_payoff, bins=60, alpha=0.7)
    plt.title("Distribution of Discounted Payoff")
    plt.xlabel("Discounted Payoff")
    plt.ylabel("Frequency")
    plt.grid(True)

    plt.tight_layout()
    plt.show()


print("\n👇 Play with Monte Carlo vs BSM (this will be a bit slower for large paths)")
interact(
    mc_vs_bsm,
    S=FloatSlider(description="S", min=10, max=300, step=5, value=100),
    K=FloatSlider(description="K", min=10, max=300, step=5, value=100),
    r=FloatSlider(description="r", min=0.0, max=0.2, step=0.01, value=0.05),
    sigma=FloatSlider(description="σ", min=0.05, max=1.0, step=0.05, value=0.2),
    T=FloatSlider(description="T", min=0.1, max=2.0, step=0.1, value=1.0),
    n_paths=IntSlider(description="Paths", min=1000, max=50000, step=1000, value=10000),
);


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00
👇 Use the sliders below to interact with S, K, r, sigma, T


interactive(children=(FloatSlider(value=100.0, description='Spot S', max=300.0, min=10.0, step=5.0), FloatSlid…


👇 Choose a ticker to compare market vs BSM (first expiry)


interactive(children=(Dropdown(description='Ticker', options=('AAPL', 'MSFT', 'TSLA', 'GOOGL', 'AMZN'), value=…


👇 Play with Monte Carlo vs BSM (this will be a bit slower for large paths)


interactive(children=(FloatSlider(value=100.0, description='S', max=300.0, min=10.0, step=5.0), FloatSlider(va…